## strassen 矩阵乘法 

In [1]:
import numpy as np

Strassen矩阵乘法能够减少“蛮力法”使用的乘法次数，当相乘的两个矩阵都是$2\times2$的规模时，可以使用如下公式代替“蛮力法”，更大规模矩阵乘法的基础：
$$\begin{equation}
\begin{aligned}
\left[
  \begin{matrix}
   c_{00} & c_{01}\\
   c_{10} & c_{11} 
  \end{matrix} 
  \right] &=
  \left[
  \begin{matrix}
   a_{00} & a_{01}\\
   a_{10} & a_{11} 
  \end{matrix} 
  \right]
  \left[
  \begin{matrix}
   b_{00} & b_{01}\\
   b_{10} & b_{11} 
  \end{matrix} 
  \right] \\
  &=\left[
  \begin{matrix}
  m_{1}+m_{4}-m_{5}+m_{7} & m_{3}+m_{5}\\
  m_{2}+m_{4} & m_{1}+m_{3}-m_{2}+m_{6}
  \end{matrix}
  \right]
  \end{aligned}
  \end{equation}
$$
其中，
$$
\begin{aligned}
m_{1} &= (a_{00}+a_{11})\times(b_{00}+b_{11})\\
m_{2} &= (a_{10}+a_{11})\times b_{00} \\
m_{3} &= a_{00} \times(b_{01}-b_{11})\\
m_{4} &= a_{11}\times(b_{10}-b_{00})\\
m_{5} &= (a_{00}+a_{01})\times b_{11} \\
m_{6} &= (a_{10}-a_{00})\times(b_{00}+b_{01})\\
m_{7} &= (a_{01}-a_{11})\times(b_{10}+b_{11})
\end{aligned}
$$  

为了确保最小规模的矩阵规模为$2\times2$,可以补充全为0的行、列使矩阵的规模为$2^{n}\times 2^{n}$,然后使用如下分块矩阵的方法递归减小矩阵规模为$2\times2$  
$$\begin{equation}
\left[
  \begin{array}{c|c}
   C_{00} & C_{01}\\ \hline
   C_{10} & C_{11} 
  \end{array} 
  \right] =
  \left[
  \begin{array}{c|c}
   A_{00} & A_{01}\\ \hline
   A_{10} & A_{11} 
  \end{array} 
  \right]
  \left[
  \begin{array}{c|c}
   B_{00} & B_{01}\\ \hline
   B_{10} & B_{11} 
  \end{array}
  \right] 
  \end{equation}
$$  
其中，矩阵$A、B、C$均是第一代矩阵，规模为$2^{n}\times 2^{n}$，经过一次“分”，形成四个规模为$2^{n-1}\times 2^{n-1}$的子问题，按照这种模式，最终只需要计算出最小规模的子问题$2\times 2$即可解决这个问题

In [3]:
def strassen_matrix(matrix_a, matrix_b, n):
    """
    type matrix_a:([[int]])
    type matrix_b:([[int]])
    type        n:int
    rtype        :([[int]])
    """
    if n == 2:
        m1 = (matrix_a[0,0]+matrix_a[1,1])*(matrix_b[0,0]+matrix_b[1,1])
        m2 = (matrix_a[1,0]+matrix_a[1,1])*matrix_b[0,0]
        m3 = matrix_a[0,0]*(matrix_b[0,1]-matrix_b[1,1])
        m4 = matrix_a[1,1]*(matrix_b[1,0]-matrix_b[0,0])
        m5 = (matrix_a[0,0]+matrix_a[0,1])*matrix_b[1,1]
        m6 = (matrix_a[1,0]-matrix_a[0,0])*(matrix_b[0,0]+matrix_b[0,1])
        m7 = (matrix_a[0,1]-matrix_a[1,1])*(matrix_b[1,0]+matrix_b[1,1])
        return np.array([[m1+m4-m5+m7,m3+m5],[m2+m4,m1+m3-m2+m6]])
    else:
        half_n = n/2
        # strassen 算法
        m1 = strassen_matrix(matrix_a[:half_n,:half_n]+matrix_a[half_n:,half_n:], \
                             matrix_b[:half_n,:half_n]+matrix_b[half_n:,half_n:], half_n)
        m2 = strassen_matrix(matrix_a[half_n:,:half_n]+matrix_a[half_n:,half_n:],matrix_b[:half_n,:half_n],half_n)
        m3 = strassen_matrix(matrix_a[:half_n,:half_n],matrix_b[:half_n,half_n:]-matrix_b[half_n:,half_n:], half_n)
        m4 = strassen_matrix(matrix_a[half_n:,half_n:],matrix_b[half_n:,:half_n]-matrix_b[:half_n,:half_n], half_n)
        m5 = strassen_matrix(matrix_a[:half_n,:half_n]+matrix_a[:half_n,half_n:],matrix_b[half_n:,half_n:],half_n)
        m6 = strassen_matrix(matrix_a[half_n:,:half_n]-matrix_a[:half_n,:half_n], \
                             matrix_b[:half_n,:half_n]+matrix_b[:half_n,half_n:], half_n)
        m7 = strassen_matrix(matrix_a[:half_n,half_n:]-matrix_a[half_n:,half_n:], \
                             matrix_b[half_n:,:half_n]+matrix_b[half_n:,half_n:], half_n)
        left  = np.concatenate((m1+m4-m5+m7,m2+m4),axis=0)
        right = np.concatenate((m3+m5,m1+m3-m2+m6),axis=0)
        
        return np.concatenate((left,right),axis=1)